<a href="https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/ForwardTacotron_HiFi_Gan_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ForwardTacotron](https://github.com/as-ideas/ForwardTacotron) and [hifi-gan](https://github.com/jik876/hifi-gan) Fine tuning notebook.

Notebook made by [rmcpantoja](https://github.com/rmcpantoja/)

## Credits:

* HiFi-GAN modification by [SortAnon](https://github.com/SortAnon/hifi-gan)
* This notebook is based on the [notebook of the FakeYou team](https://colab.research.google.com/github/justinjohn0306/FakeYou-Tacotron2-Notebook/blob/main/FakeYou_HiFi_GAN_Fine_Tuning.ipynb) by `Cookie PPP` and `Justinjohn03`.

## First steps

In [ ]:
#@title Check GPU
#@markdown You need a tesla t4 minimum.
!nvidia-smi

In [ ]:
#@title Mount Google Drive
#@markdown This is useful to import data needed for fine tuning in HiFi-GAN and export the training of these fine tuned models.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title Install software
#@markdown This cell installs ForwardTacotron, HiFi-GAN and some dependencies needed to execute the tasks.
#@markdown #### Use version 3.2 of ForwardTacotron?

#@markdown Version 3.2 is the last to contain only single speaker support.
only_singlespeaker_version = True #@param {type:"boolean"}
#@markdown ---
%cd /content
!git clone https://github.com/as-ideas/ForwardTacotron.git
!git clone https://github.com/SortAnon/hifi-gan.git
# rollback to last single speaker version (tag v3.2)
%cd ./ForwardTacotron
if only_singlespeaker_version.
  !git checkout 632b453c6cb6d15dfe4dd168cd60c60e56731829
!rm -r .git/
# install
!apt-get install espeak-ng
!pip install numba==0.49.1 librosa==0.7.2 pyworld phonemizer webrtcvad PyYAML soundfile scipy tensorboard matplotlib unidecode inflect
!pip install --upgrade gdown

## Data and training

In [ ]:
#@title set data paths
!mkdir /content/ForwardTacotron/wavs
#@markdown #### Preprocessed dataset path (data and config):
project = "/content/drive/MyDrive/auronsesion.zip" #@param {type:"string"}
print("Extracting pre-processed dataset...")
!unzip -q "$project" -d /content/ForwardTacotron
#@markdown ---
#@markdown #### Audio dataset path (in zip format):
audio_dataset = "/content/drive/MyDrive/Fakeyou/AURON/wavs.zip" #@param {type:"string"}
print("Extracting audio...")
!unzip -q "$audio_dataset" -d /content/ForwardTacotron/wavs
#@markdown ---
#@markdown #### Folder where the Tacotron or ForwardTacotron checkpoints are located
checkpoints_dir = "/content/drive/MyDrive/ForwardTacotron/checkpoints" #@param {type:"string"}
!cp -r "$checkpoints_dir" /content/ForwardTacotron
#@markdown ---
#@markdown #### Paths of the train and validation filelists
train_filelist = "/content/drive/MyDrive/Fakeyou/AURON/list.txt" #@param {type:"string"}
val_filelist = "/content/drive/MyDrive/Fakeyou/AURON/list.txt" #@param {type:"string"}
#@markdown ---


In [ ]:
#@title Create ground truth aligned Dataset
#@markdown With this dataset, HiFi-GAN can distinguish the sound of the model to be aligned with the original wavs to adapt their sound.

#@markdown ---
%cd /content/ForwardTacotron
# create GTA features for:
#@markdown #### Select the model you would like to work on:
GTA_model = "forward" #@param ["forward", "tacotron"]
#@markdown ---
!python train_{GTA_model}.py --force_gta

In [ ]:
#@title Train HiFi-GAN
%cd /content/hifi-gan

import os
import shutil
#@markdown #### Desired name for the model
model_name = "auron" #@param {type:"string"}
#@markdown ---
#@markdown #### Output directory. (recommended to save it on drive)
output_dir = "/content/drive/My Drive/forward_finetune"  #@param {type:"string"}
savedir = output_dir+"/"+model_name
#@markdown ---
#@markdown #### Step interval to save models
checkpoint_interval = 250 #@param {type:"integer"}
#@markdown ---
import gdown
d = 'https://drive.google.com/uc?id='
os.chdir('/content/hifi-gan')
# copy fine tune dataset:
if os.path.exists("/content/hifi-gan/ft_dataset"):
    shutil.rmtree("/content/hifi-gan/ft_dataset")
shutil.copytree("/content/ForwardTacotron/data/gta", "/content/hifi-gan/ft_dataset/wavs/")
if os.path.exists("/content/hifi-gan/wavs"):
    shutil.rmtree("/content/hifi-gan/wavs")
shutil.copytree("/content/ForwardTacotron/wavs/", "/content/hifi-gan/wavs/")
shutil.copyfile(train_filelist, "/content/hifi-gan/training.txt")
shutil.copyfile(val_filelist, "/content/hifi-gan/validation.txt")

if not os.path.exists(savedir):
    os.makedirs(savedir)
if not os.path.exists(os.path.join(savedir, "do_00000000")):
    print("Downloading universal model...")
    gdown.download(d+"1O63eHZR9t1haCdRHQcEgMfMNxiOciSru", os.path.join(savedir, "do_00000000"), quiet=False)
    gdown.download(d+"1qpgI41wNXFcH-iKq1Y42JlBC9j0je8PW", os.path.join(savedir, "g_00000000"), quiet=False)
    start_from_universal = "--warm_start True "
else:
    start_from_universal = ""


!python train.py --fine_tuning True --config config_v1b.json \{start_from_universal}\
--checkpoint_interval {checkpoint_interval} --checkpoint_path "{savedir}" \
--input_training_file "/content/hifi-gan/training.txt" \
--input_validation_file "/content/hifi-gan/validation.txt" \
--input_wavs_dir "" --input_mels_dir "ft_dataset"


